# Read a MySQL DataBase and generate specific output
Hint: Works with real index

In [126]:
from database_to_pandas import *
import sys

In [127]:
import pandas as pd

In [128]:
df=pd.read_json('data/db.json')
df=df.reset_index(drop=True)

In [129]:
dfid=df[df['id']==id].index.values[0]

In [130]:
df.shape[0]

308

In [131]:
NEW=True
if NEW:
    dfid=df.shape[0]
    df=df.append(df.loc[[dfid-1]]).reset_index(drop=True)

In [132]:
df.loc[dfid]

address                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
body           \n    Solicito la inscripción del Proyecto ``MÉTRICAS DE VINCULACIÓN UNIVERSIDAD ENTORNO ETAPA 3.'' \n    con una duración de 8 mesesy el cual se encuentrá financiado por varias instancias de la Universidad. \n    El proyecto se encuentra ya diligenciado en la plataforma SIIU de acuerdo al documento anexo. \n    El equipo de trabajo es el siguiente: \n    \begin{itemize}\n    \item Diego Restrepo, Investigador Principal, 5h/semana\n    \item Alejandro Uribe Tirado, 

In [133]:
body=r'''
    Solicito una prorroga por 6 meses para la ejecución de la Estrategía de Sostenibilidad del
    Grupo de Fenomenología de Interacciones Fundamentales, 2018-2020 debido a los problemas de
    gestión de los recursos originados por la pandemía de Covid-19.
'''

In [134]:
EDIT=True
if EDIT:
    df.loc[dfid,'description']=r'Inscripción de proyecto'
    df.loc[dfid,'para']=r'''
    Señores\\
    Comité Técnico\\
    Facultad de Ciencias Exactas y Naturales
    '''    
    df.loc[dfid,'body']=r'{}'.format(body)
    df.loc[dfid,'closing']=r'Agradeciendo su colaboración en este sentido'

In [135]:
df.loc[dfid,'preamble']=r'''
\usepackage{udea}
%\usepackage[spanish]{babel}
'''

In [136]:
df.iloc[-1:]

,address,body,closing,description,fecha,id,link,opening,para,preamble,signature
308,,"\n Solicito una prorroga por 6 meses para la ejecución de la Estrategía de Sostenibilidad del\n Grupo de Fenomenología de Interacciones Fundamentales, 2018-2020 debido a los problemas de\n gestión de los recursos originados por la pandemía de Covid-19.\n",Agradeciendo su colaboración en este sentido,Inscripción de proyecto,1595030400000,307,http://gfif.udea.edu.co/cartas/cartas.php?step=1&id=,Respetados señores:,\n Señores\\\n Comité Técnico\\\n Facultad de Ciencias Exactas y Naturales\n,\n\usepackage{udea}\n%\usepackage[spanish]{babel}\n,Diego_en_sign


In [137]:
df["id"]=df.index

In [138]:
df.to_json('data/db.json',orient='records',force_ascii=False)

In [139]:
import getpass
import pymysql
import unidecode
import pandas as pd
import tempfile
import subprocess
import sys
from latexfiles import *


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth',500)

def generate_pdf(latexfile,
    pdf='output.pdf',
    dwld_dir='/home/restrepo/downloads',
    url_dir='file:///home/restrepo/downloads',id=None):
    """
    pdflatex upon latexfile to generate 'pdf'  file in 
    'dwld_dir' with browser link 'url_dir'
    """

    f=tempfile.NamedTemporaryFile('w',suffix='.tex',delete=False)
    f.write(latexfile)
    f.close()
    lo=subprocess.Popen('pdflatex {:s}'.format(f.name).split(),cwd='/tmp',
                        stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()
    pdffile='{:s}'.format(f.name).split('.tex')[0]+'.pdf'
    if lo[0].decode('utf8').find(pdffile.split('/')[-1])>-1:
        
        kk=subprocess.Popen('mv {:s} {:s}/{:s}'.format(pdffile,dwld_dir,pdf).split()).communicate()
        print('output in {:s}/{:s}'.format(url_dir,pdf))
        for suffix in  ['.aux','.log','.out','.tex']:
            sc=subprocess.Popen( ['rm', '{:s}'.format(f.name).split('.tex')[0]+suffix] ).communicate()
    else:
        print(lo[0].decode('utf8'))
        sys.exit('LaTeX ERROR:')    
    return latexfile

class database_to_pandas(object):
    """
    Convert Data Base  to pandas and filtered index to Series and 
    define methods to use them.
    Intialize with choose_db and main_table to build attibute: choose_db[main_table]
    See for example:
    {'Diego_Restrepo':'cartas','cartas_gfif':'cartas','seminarios':'talks'}
    The optional aux_tables must be set to '' if not required, otherwise 
    'cartas_sign' is used
    udea.sty and udea.pdf must be instaled
    """
    def __init__(self,host='localhost',user='root',choose_db='Diego_Restrepo',main_table='cartas',id=None,
                 aux_tables='cartas_sign',common_columns='signature',DB=True,verbose=False):
        if DB:
            password=getpass.getpass()
            self.choose_db={choose_db:main_table}
            self.DataBase=choose_db
            conn = pymysql.connect(host=host, port=3306, user=user, passwd=password, db=choose_db)
            df=pd.read_sql('select * from {};'.format(self.choose_db[choose_db]), con=conn)
        else:
            df=pd.read_json('data/db.json')
        #TODO: generilize to addtional tables
        if aux_tables:
            if DB:
                sg=pd.read_sql('select * from {};'.format(aux_tables), con=conn)
                conn.close()
            else:
                sg=pd.read_json('data/sg.json')

        self.pandas_DataFrame=df
        self.pandas_signature=sg
        #print(df.shape[0])
        #return df,sg
        if df.shape[0]:
            if not id:
                id=df.id.values[-1]
            self.pandas_Series=df[df.id==id].reset_index(drop=True).loc[0]
            if aux_tables:
                #TODO:
                self.pandas_Series['common_{}'.format(common_columns)]=sg[
                    sg[common_columns]==self.pandas_Series[common_columns]].reset_index(drop=True).loc[0,'sign']
            if verbose:
                print(self.pandas_Series)
        else:
            sys.exit('Error reading databases')
            return df

    
    def to_latex(self,func):
        '''Build a function to build the latex file with the self.Series keys
           and call it from here
        '''
        #TODO: check if class has methoed
        self.latexfile=func(self.pandas_Series)
        return self.latexfile
    #return generate_pdf(c)
    def to_pdf(self,func,**kwargs):
        '''Build a function to build the latex file with the self.Series keys
           and call it from here
        '''
        self.latexfile=func(self.pandas_Series)
        return generate_pdf(self.latexfile,**kwargs)
        
    def backup_database(self):
        '''
        Backup DB
        '''
    
        password=getpass.getpass()

        print('Trying to create backup file db_{:s}_backup.sql'.format( self.DataBase  )   )
        ok=subprocess.Popen('mysqldump -u root -p{:s} {:s} > db_{:s}_backup.sql'.format(
                         password,self.DataBase,self.DataBase),shell=True,
                         stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

        print()

        return ok
import getpass

In [143]:
id=308

In [144]:
data_base='Diego_Restrepo'
db=database_to_pandas(choose_db=data_base,main_table='cartas',id=id,DB=False)

In [145]:
latexfile=db.to_pdf(cartas,pdf='cartas.pdf',
            dwld_dir='/home/restrepo/downloads',
            url_dir='http://localhost/downloads',id=id)

output in http://localhost/downloads/cartas.pdf


In [30]:
id=None # last
#id=176 #Invitation letter
#id=281 # Invitado Nacional
#id=289 # Super recommendation corrected
#id=283 #certificado estancia
#id=307 # Generic Recommmendation letter
#id=306 #Generic Aval horas de investigación
#id=105
data_base='Diego_Restrepo' #Diego_Restrepo;cartas_gfif;seminarios
#data_base='cartas_gfif' #Diego_Restrepo;cartas_gfif;seminarios
try:
    if id:
        db=database_to_pandas(choose_db=data_base,main_table='cartas',id=id)
    else:    
        db=database_to_pandas(choose_db=data_base,main_table='cartas')
except KeyError:
    sys.exit('Uncomment Personal Data Base')       

KeyboardInterrupt: Interrupted by user

In [ ]:
if id:
    latexfile=db.to_pdf(cartas,pdf='cartas.pdf',
            dwld_dir='/home/restrepo/downloads',
            url_dir='http://fisica.udea.edu.co/downloads',id=id)
else:
    latexfile=db.to_pdf(cartas,pdf='cartas.pdf',
            dwld_dir='/home/restrepo/downloads',
            url_dir='http://fisica.udea.edu.co/downloads')

### Backup database

In [ ]:
db.pandas_DataFrame.to_json('data/db.json',orient='records',force_ascii=False)
db.pandas_signature.to_json('data/sg.json',orient='records',force_ascii=False)
db.backup_database()

In [ ]:
!git commit -am 'fix signature json' > /dev/null
!git push origin master > /dev/null

In [ ]:
db.pandas_DataFrame.loc[300]

In [ ]:
print( db.pandas_DataFrame.loc[300,'para'] )

In [ ]:
print( db.pandas_DataFrame.loc[300,'body'] )

\def\proyecto{
%1: de movilidad Sao Paulo - Universidad de Antioquia  ``SPRINT – 3rd Edition 2019''
%2:``FAPESP - UNIVERSIDAD DE ANTIOQUIA 2019''
%3: MÉTRICAS DE VINCULACIÓN UNIVERSIDAD ENTORNO ETAPA 3
%4:
``Desarrollo de algoritmos para la detección de neumonía y SARS-Cov-2 usando imágenes de Rayos-X.''
}

Solicito el aval para enviar el proyecto  \proyecto 
y del cual soy el investigador principal. Dicho proyecto se realiza en el marco del  
Fondo de apoyo a los grupos de trabajo de la iniciativa \#UdeA responde al Covid-19.
Dicho proyecto se hace en conjunto con la Facultad de Medicina y el CERN a través de un
convenio interinsitucional con la dedicación de 6 horas semanalas del coinvestigador del CERN: Omar Zapata.

La duración del proyecto será de 9 meses. 

Se adjunta el borrador de la propuesta en trámite y el presupuesto

### Other databases

In [ ]:
dbs=database_to_pandas(choose_db='seminarios',main_table='talks',aux_tables='')

In [ ]:
lfs=dbs.to_pdf(seminarios,pdf='seminarios.pdf',
        dwld_dir='/home/restrepo/downloads',
        url_dir='http://gfif.udea.edu.co/downloads')

### Get DataFrame

In [ ]:
db.pandas_DataFrame.to_excel('kk.xlsx',index=False)

## Manual

In [ ]:
latexfile

In [ ]:
password=getpass.getpass()

In [ ]:
choose_db=0
tipo_carta=['Diego_Restrepo', 'cartas_gfif']

conn = pymysql.connect(host='localhost', port=3306, user='root', passwd=password, db=tipo_carta[choose_db])

df=pd.read_sql('select * from cartas;', con=conn)

sg=pd.read_sql('select * from cartas_sign;', con=conn)

In [ ]:
conn.close()Implicaciones de modelos de materia oscura en experimentos de detección directa

In [ ]:
df=df[df.body.str.lower().str.contains('editor')]

In [ ]:
df=df[df.id==281]

Last entry

In [ ]:
df

In [ ]:
c=df[df.id==df.id.values[-1]].reset_index(drop=True).loc[0]
c['sign']=sg[sg.signature==c.signature].reset_index(drop=True).loc[0,'sign']

In [ ]:
c

In [ ]:
row=c

In [ ]:
lo=generate_pdf(row)

In [ ]:
print(lo[0].decode('utf8'))Implicaciones de modelos de materia oscura en experimentos de detección directa

In [ ]:
c.sign

In [ ]:
ok

In [ ]:
'mysqldump -u root -p{:s} {:s} > db_{:s}_backup.sql'.format(password,tipo_carta[NTC],tipo_carta[NTC]).split()

## Search in body

In [ ]:
id=99
if id >0:
    dfs=df[df.id==id].reset_index(drop=True)
else:    
    dfs=df[df.body.str.lower().map(unidecode.unidecode).str.contains('impacto social')].reset_index(drop=True)

In [ ]:
dfs

In [ ]:
c=dfs.loc[dfs.index.values[-1]]

In [ ]:
c

In [ ]:
c.keys().values

In [ ]:
for k in c.keys():
    print(k)
    if c[k]==None:
        c[k]=''Implicaciones de modelos de materia oscura en experimentos de detección directa

In [ ]:
c.closing

In [ ]:
c.closing

In [ ]:
latexfile=r'''
%'''+c.description+r'''
\documentclass{letter}
\usepackage[utf8]{inputenc}
'''+c.preamble+r'''
'''+c.address+r'''
\signature{'''+sg[sg.signature==c.signature].sign.values[0]+r'''}
\begin{document}
\begin{letter}{'''+c.para+r'''}

\opening{'''+c.opening+r'''} 

'''+c.body+r'''

\closing{'''+c.closing+r'''}
\end{letter}
\end{document}
'''

In [ ]:
print(latexfile)

In [ ]:
f=tempfile.NamedTemporaryFile('w',suffix='.tex',delete=False)

In [ ]:
f.write(latexfile)

In [ ]:
f.name

In [ ]:
f.close()

In [ ]:
lo=subprocess.Popen('pdflatex {:s}'.format(f.name).split(),cwd='/tmp',stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

In [ ]:
pdffile='{:s}'.format(f.name).split('.tex')[0]+'.pdf'
if lo[0].decode('utf8').find(pdffile.split('/')[-1])>-1:
    pdf='carta_{:d}.pdf'.format(c['id'])
    dwld_dir='downloads'
    kk=subprocess.Popen('mv {:s} /var/www/{:s}/{:s}'.format(pdffile,dwld_dir,pdf).split()).communicate()
    print('output in http://gfif.udea.edu.co/{:s}/{:s}'.format(dwld_dir,pdf))
    for suffix in  ['.aux','.log','.out','.tex']:
        subprocess.Popen( ['rm', '{:s}'.format(f.name).split('.tex')[0]+suffix] ).communicate()
else:
    sys.exit('LaTeX ERROR:')    

In [ ]:
%pycat database_to_pandas.py

In [ ]:
r'https://scholar.google.com/scholar?hl=en&q=doi'